In [ ]:
%%capture
!pip install pymongo
!pip install pycld2
!pip install python-rake

In [1]:
from __future__ import print_function

import string

import json
import os
from os import getcwd, pardir
from os import listdir
from os.path import isfile, join
from glob import glob

from utils.literature import DataLoader, get_files, get_full_text, get_document_title, is_english
from utils.processing import create_chunks

import RAKE

import pymongo
from pymongo import MongoClient as DBClient

from multiprocessing.pool import Pool
from tqdm._tqdm_notebook import tqdm

import pycld2 as cld2

In [2]:
root_dir = join(pardir, 'dataset')
files = get_files(root_dir)
chunks = create_chunks(files)

In [8]:
def process_chunk(args):
    with DBClient('localhost', 27017, w=0) as client:

        #open a connection to the database
        db = client['covid_19']
        keyphrase_index_collection = db['keyphrase_index']
        
        db_documents = []
        rake = RAKE.Rake(RAKE.SmartStopList())

        #iterate over all documents in chunk
        for fpath, doc_id in args:
            doc_id = int(doc_id)
            data_loader = DataLoader(fpath)

            doc_title = get_document_title(fpath, data_loader)
            text = get_full_text(fpath, data_loader)

            if doc_title == '' or not is_english(doc_title):
                continue
        
            keyphrases = rake.run(text, minCharacters = 5, maxWords = 3, minFrequency = 10)
            keyphrases = keyphrases[:10]


            keyphrases_json = [{'keyphrase': keyphrase[0], 'score': keyphrase[1]} for keyphrase in keyphrases]
            db_documents.append({'_id': doc_id, 'keyphrases': keyphrases_json})

        keyphrase_index_collection.insert_many(db_documents)

In [9]:
pool = Pool()
chunks = create_chunks(files)

for _ in tqdm(pool.imap_unordered(process_chunk, chunks), total=len(chunks)):
    pass

pool.close()
pool.join()